# Lab | Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')

In [5]:
#!pip install pandas

In [4]:
import pandas as pd
df = pd.read_csv('data/Data.csv')

In [5]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


## LLMChain

In [16]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain

In [7]:
#Replace None by your own value and justify
llm = ChatOpenAI(temperature=0.7)

In [12]:
prompt = ChatPromptTemplate.from_template( 
  "Describe following product: {product}:"
)

In [10]:
chain = LLMChain(llm=llm, prompt=prompt)

C:\Users\hibi9\AppData\Local\Temp\ipykernel_28156\1929250370.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)


In [11]:
product = "Pillows Insert"
chain.run(product)

C:\Users\hibi9\AppData\Local\Temp\ipykernel_28156\179363538.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain.run(product)


'A pillow insert is a cushioned inner piece that is placed inside a pillow cover to provide structure and support to the pillow. It is typically filled with materials such as polyester fiberfill, down feathers, or memory foam to provide comfort and shape to the pillow. Pillow inserts come in various sizes and shapes to fit different pillow covers and are often used to refresh or customize the look of a pillow.'

## SimpleSequentialChain

In [ ]:
from langchain.chains import SimpleSequentialChain

In [13]:
llm = ChatOpenAI(temperature=0.9)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "Describe following product: {product}:"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [19]:

# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Please give a better product as {product}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [20]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [21]:
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
Pillow insert is a rectangular or square-shaped filling that goes inside a pillow cover to provide support and comfort when used as a cushion for resting or sleeping. It is typically made of materials such as polyester, down feathers, or memory foam, and comes in various sizes to fit different pillow covers. The insert can be removed and replaced as needed to maintain the shape and firmness of the pillow.
A better product could be a customizable pillow insert that allows users to adjust the firmness and support level to their liking. This could be achieved through different compartments filled with materials of varying densities, allowing for a personalized sleeping experience. Additionally, the insert could have a breathable and hypoallergenic cover to ensure comfort and hygiene. This innovative design would provide a superior level of customization and comfort compared to traditional pillow inserts.

> Finished chain.


'A better product could be a customizable pillow insert that allows users to adjust the firmness and support level to their liking. This could be achieved through different compartments filled with materials of varying densities, allowing for a personalized sleeping experience. Additionally, the insert could have a breathable and hypoallergenic cover to ensure comfort and hygiene. This innovative design would provide a superior level of customization and comfort compared to traditional pillow inserts.'

**Repeat the above twice for different products**

In [22]:
overall_simple_chain.run("Luxury Air Mattress")



> Entering new SimpleSequentialChain chain...
The luxury air mattress is a high-end, premium quality bedding product designed for superior comfort and support. It is made with top-of-the-line materials that provide a plush and cushioned sleeping surface. The mattress is equipped with advanced air technology that allows users to adjust the firmness levels to their desired comfort level. It also features a built-in pump for easy inflation and deflation, as well as a durable construction that ensures long-lasting performance. The luxury air mattress is perfect for those seeking a luxurious sleeping experience while camping, traveling, or as an extra bed for guests.
The DreamCloud Luxury Hybrid Mattress is a top-of-the-line bedding product that provides unparalleled comfort and support. Made with premium materials including gel-infused memory foam and individually wrapped coils, this mattress offers a plush and cushioned sleeping surface that contours to your body for ultimate comfort.



"The DreamCloud Luxury Hybrid Mattress is a top-of-the-line bedding product that provides unparalleled comfort and support. Made with premium materials including gel-infused memory foam and individually wrapped coils, this mattress offers a plush and cushioned sleeping surface that contours to your body for ultimate comfort.\n\nThe DreamCloud Luxury Hybrid Mattress features advanced technology that allows you to customize the firmness levels to your liking, ensuring a perfect night's sleep every time. With a built-in pump for easy inflation and deflation, this mattress is not only luxurious but also convenient.\n\nConstructed with durability in mind, the DreamCloud Luxury Hybrid Mattress is built to last, providing long-lasting performance for years to come. Whether you're camping, traveling, or hosting guests, this mattress is the perfect choice for those seeking a luxurious sleeping experience."

In [23]:
overall_simple_chain.run("Butterbrezel")



> Entering new SimpleSequentialChain chain...
Butterbrezel is a traditional German snack made of a soft pretzel that is coated with a generous amount of butter. The pretzel is typically warm and chewy, with a crispy exterior and a soft, fluffy interior. The butter adds a rich and savory flavor to the pretzel, making it a delicious and indulgent treat. Butterbrezel is often enjoyed as a snack or as a light meal on the go.
One delicious alternative to Butterbrezel is a gourmet filled pretzel. Imagine a soft, freshly baked pretzel filled with gooey mozzarella cheese, savory pepperoni, and zesty marinara sauce for a mouthwatering twist on a classic snack. This filled pretzel is baked to perfection, creating a crispy exterior and a warm, flavorful interior that will satisfy your cravings for something savory and satisfying. Perfect for enjoying on the go or as a satisfying snack at home, this gourmet filled pretzel is sure to become your new favorite indulgence.

> Finished chain.


'One delicious alternative to Butterbrezel is a gourmet filled pretzel. Imagine a soft, freshly baked pretzel filled with gooey mozzarella cheese, savory pepperoni, and zesty marinara sauce for a mouthwatering twist on a classic snack. This filled pretzel is baked to perfection, creating a crispy exterior and a warm, flavorful interior that will satisfy your cravings for something savory and satisfying. Perfect for enjoying on the go or as a satisfying snack at home, this gourmet filled pretzel is sure to become your new favorite indulgence.'

## SequentialChain

In [37]:
from langchain.chains import SequentialChain

In [71]:
llm = ChatOpenAI(temperature=0.9)

first_prompt = ChatPromptTemplate.from_template(
  "Translate this review into german: {review}"
)

chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key='review1' #Give a name to your output
                    )

In [72]:
second_prompt = ChatPromptTemplate.from_template(
    "Summarize this german review: {review1}"
)

chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key='review2' #give a name to this output
                    )


In [73]:
# prompt template 3: translate to english or other language
third_prompt = ChatPromptTemplate.from_template(
    "Translate the summarized review '{review2}' into Japanese"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="review3"
                      )


In [74]:

# prompt template 4: follow up message that take as inputs the two previous prompts' variables
fourth_prompt = ChatPromptTemplate.from_template(
        "Check if '{review3}' a Japanese translation of '{review2}'?'"
)
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="review4"
                     )


In [75]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message

overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=['review'],
    output_variables=['review1', 'review2', 'review3'],
    verbose=True
)

In [78]:
df.Review[1]

'I loved the waterproof sac, although the opening was made of a hard plastic. I don’t know if that would break easily. But I couldn’t turn my phone on, once it was in the pouch.'

In [80]:
review = df.Review[1]
overall_chain(review)



> Entering new SequentialChain chain...

> Finished chain.


{'review': 'I loved the waterproof sac, although the opening was made of a hard plastic. I don’t know if that would break easily. But I couldn’t turn my phone on, once it was in the pouch.',
 'review1': 'Ich mochte die wasserdichte Tasche, obwohl die Öffnung aus hartem Kunststoff gefertigt war. Ich weiß nicht, ob das leicht brechen würde. Aber ich konnte mein Telefon nicht einschalten, sobald es sich in der Hülle befand.',
 'review2': 'The reviewer liked the waterproof bag, although they were concerned about the opening being made of hard plastic. They were unsure if it would easily break. Additionally, they had trouble turning on their phone while it was inside the case.',
 'review3': 'レビュアーは防水バッグが好きだったが、開口部が硬いプラスチックで作られていることに懸念を抱いていた。すぐに壊れるかどうかわからなかった。さらに、ケースの中に入れたまま電話をオンにするのに苦労していた。'}

**Repeat the above twice for different products or reviews**

## Router Chain

In [82]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [87]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

In [83]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [84]:
llm = ChatOpenAI(temperature=0)

In [88]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [89]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [90]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [91]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [92]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

C:\Users\hibi9\AppData\Local\Temp\ipykernel_28156\2048584746.py:1: LangChainDeprecationWarning: Use RunnableLambda to select from multiple prompt templates. See example in API reference: https://api.python.langchain.com/en/latest/chains/langchain.chains.router.multi_prompt.MultiPromptChain.html
  chain = MultiPromptChain(router_chain=router_chain,


In [93]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


"Black body radiation refers to the electromagnetic radiation emitted by a perfect black body, which is an idealized physical body that absorbs all incident electromagnetic radiation and emits radiation at all frequencies. The radiation emitted by a black body depends only on its temperature and follows a specific distribution known as Planck's law. This type of radiation is important in understanding concepts such as thermal radiation and the behavior of objects at different temperatures."

In [94]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


'The answer to 2 + 2 is 4.'

In [96]:
chain.run("Why is the GPU better than the CPU to train models?")



> Entering new MultiPromptChain chain...
computer science: {'input': 'Why is the GPU better than the CPU to train models?'}
> Finished chain.


"The GPU is better than the CPU for training models because of its parallel processing capabilities. GPUs have thousands of cores that can handle multiple tasks simultaneously, making them much faster at performing matrix operations and other computations required for training machine learning models. This parallel processing power allows GPUs to significantly reduce the time it takes to train models compared to CPUs, which have fewer cores and are better suited for sequential processing tasks. Additionally, GPUs are optimized for handling large amounts of data and complex mathematical calculations, making them ideal for training deep learning models that require intensive computations. Overall, the GPU's ability to handle parallel processing makes it a more efficient and effective choice for training models compared to the CPU."

**Repeat the above at least once for different inputs and chains executions - Be creative!**

In [117]:
florist_template = """ You have an incredible eye for design. You're a master \
at arranging flowers, creating stunning bouquets and arrangements that \
convey the perfect emotion. You achieve this by skillfully considering \
the individual characteristics of each flower – its color, shape, and \
texture – and then artfully combining them to create a harmonious and expressive whole.

Here is a question:
{input}"""

marketing_template = """ You as a Marketing Guru have a talent for connecting with people. \
You're a skilled communicator, able to identify and understand the \
needs of your audience. You excel at crafting compelling messages \
that resonate with potential customers, by carefully analyzing their \
interests and motivations, and then tailoring your approach to effectively reach them.

Here is a question:
{input}"""

developer_template = """ You are one of the best developers of the world. \
You have an excellent knowledge of and understanding of code \
in mostly all languages. \
You have the ability to think, reflect, debate, discuss and \
evaluate why certain code is good or bad. Or what code is correct to solve a problem.\
You mostly use ruby to explain things, as it's the best development language in your opinion

Here is a question:
{input}"""

astronaut_template = """ You possess extraordinary courage and a thirst for exploration. \
You're a skilled astronaut, capable of operating complex spacecraft and conducting crucial \
research in the challenging environment of space. You achieve this by meticulously mastering \
intricate technical skills, maintaining unwavering focus in demanding situations, and \
collaborating effectively with your crewmates to accomplish ambitious missions beyond Earth.

Here is a question:
{input}"""

In [108]:
prompt_infos = [
    {
        "name": "florist", 
        "description": "Good for answering questions about flowers", 
        "prompt_template": florist_template
    },
    {
        "name": "marketing", 
        "description": "Good for answering marketing questions", 
        "prompt_template": marketing_template
    },
    {
        "name": "developer", 
        "description": "Good for answering questions about development", 
        "prompt_template": developer_template
    },
    {
        "name": "astronaut", 
        "description": "Good for answering questions about the space and spaceships", 
        "prompt_template": astronaut_template
    }
]

In [118]:
llm = ChatOpenAI(temperature=0)

In [120]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain

destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)
destinations_str

'florist: Good for answering questions about flowers\nmarketing: Good for answering marketing questions\ndeveloper: Good for answering questions about development\nastronaut: Good for answering questions about the space and spaceships'

In [121]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [123]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [124]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

In [125]:
chain.run("what greenery can be used to create a boucet")



> Entering new MultiPromptChain chain...
florist: {'input': 'what greenery can be used to create a bouquet'}
> Finished chain.


'There are many types of greenery that can be used to create a beautiful bouquet. Some popular options include eucalyptus, ferns, ivy, salal, and ruscus. These greenery options can add texture, depth, and contrast to a bouquet, enhancing the overall look and feel of the arrangement. Additionally, greenery can help to fill out a bouquet and provide a lush backdrop for the flowers. Experimenting with different types of greenery can help you create a unique and visually appealing bouquet.'

In [126]:
chain.run("How to develop an API to Mailgun?")



> Entering new MultiPromptChain chain...
developer: {'input': 'How to develop an API to Mailgun?'}
> Finished chain.


"To develop an API to Mailgun using Ruby, you can follow these steps:\n\n1. First, you need to sign up for a Mailgun account and obtain your API key and domain name.\n\n2. Install the `mailgun-ruby` gem by adding it to your Gemfile and running `bundle install`.\n\n3. Create a new Ruby file (e.g. `mailgun_api.rb`) and require the `mailgun-ruby` gem at the top of the file.\n\n4. Initialize the Mailgun client with your API key and domain name:\n\n```ruby\nrequire 'mailgun-ruby'\n\nmailgun = Mailgun::Client.new(api_key: 'YOUR_API_KEY', domain: 'YOUR_DOMAIN_NAME')\n```\n\n5. Now you can start using the Mailgun API to send emails, manage mailing lists, and more. Here's an example of sending an email using the Mailgun API:\n\n```ruby\nmailgun.messages.send_email(\n  from: 'you@example.com',\n  to: 'recipient@example.com',\n  subject: 'Hello from Mailgun',\n  text: 'This is a test email sent using the Mailgun API'\n)\n```\n\n6. You can also use the Mailgun API to track email deliveries, bounce

In [127]:
chain.run("Where to out a video to rich the audience of hotel owners?")



> Entering new MultiPromptChain chain...
marketing: {'input': 'Where to put a video to reach the audience of hotel owners?'}
> Finished chain.


"To reach the audience of hotel owners, you should consider placing the video on platforms that are commonly used by this demographic. Some effective options may include:\n\n1. LinkedIn: Many hotel owners are active on LinkedIn, as it is a professional networking platform. You can share the video on your company page or in relevant industry groups to reach this audience.\n\n2. Hotel industry websites and forums: Consider reaching out to popular hotel industry websites and forums to see if they would be willing to feature your video. This can help you reach a targeted audience of hotel owners who are actively engaged in the industry.\n\n3. Email marketing: If you have a database of hotel owners' email addresses, consider sending out a targeted email campaign that includes a link to the video. This can help you reach a specific audience and drive engagement with your content.\n\n4. Social media advertising: Utilize social media advertising platforms such as Facebook, Instagram, or Twitte

In [128]:
chain.run("Why it takes so long to travel to mars?")



> Entering new MultiPromptChain chain...
astronaut: {'input': 'Why does it take so long to travel to Mars?'}
> Finished chain.


'Traveling to Mars takes a long time primarily due to the vast distance between Earth and Mars. The average distance between the two planets is about 225 million kilometers, but this distance can vary depending on their positions in their respective orbits. \n\nAdditionally, the speed at which spacecraft can travel is limited by the amount of fuel they can carry and the technology available for propulsion. Current spacecraft technology, such as chemical propulsion systems, can only travel at a certain speed, which means it takes several months to reach Mars.\n\nFurthermore, the journey to Mars must be carefully planned to take advantage of the most efficient trajectory, known as a Hohmann transfer orbit, which minimizes the amount of fuel needed for the journey. This requires precise calculations and timing to ensure the spacecraft arrives at Mars when it is closest to Earth in its orbit.\n\nOverall, the combination of the vast distance between Earth and Mars, the limitations of curren

In [129]:
chain.run("Why it seems that the moon is really huge sometimes?")



> Entering new MultiPromptChain chain...
astronaut: {'input': 'Why does the moon appear larger at certain times?'}
> Finished chain.


'The moon appears larger at certain times due to an optical illusion known as the "moon illusion." This phenomenon occurs when the moon is near the horizon, where there are objects like trees, buildings, or mountains for reference. When the moon is closer to the horizon, our brain perceives it to be larger because it is being compared to familiar objects in the foreground. In reality, the size of the moon remains constant throughout its orbit around Earth.'